In [23]:
!pip install opendatasets
!pip install pandas

In [24]:
import opendatasets as od
import pandas

od.download(
    "https://www.kaggle.com/datasets/tourist55/alzheimers-dataset-4-class-of-images")


Skipping, found downloaded files in "./alzheimers-dataset-4-class-of-images" (use force=True to force download)


In [25]:
import pandas as pds
import os
import cv2
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D,DepthwiseConv2D, Flatten, Dense, BatchNormalization, Dropout, GlobalAveragePooling2D
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from keras.metrics import Precision, Recall, AUC
import keras.backend as K
import math

In [26]:
dataset_dir = '/content/alzheimers-dataset-4-class-of-images/Alzheimer_s Dataset'

categories = ['VeryMildDemented', 'MildDemented', 'ModerateDemented', 'NonDemented']

data = []
labels = []

for category in categories:
    path = os.path.join(dataset_dir, 'train', category)
    class_num = categories.index(category)

    for img in os.listdir(path):
        img_array = cv2.imread(os.path.join(path, img), cv2.IMREAD_COLOR)
        img_array = cv2.resize(img_array, (100, 100))  # Resize the image
        # img_array = img_array
        data.append([img_array, class_num])

for category in categories:
    path = os.path.join(dataset_dir, 'test', category)
    class_num = categories.index(category)

    for img in os.listdir(path):
        img_array = cv2.imread(os.path.join(path, img), cv2.IMREAD_COLOR)
        img_array = cv2.resize(img_array, (100, 100))  # Resize the image
        # img_array = img_array
        data.append([img_array, class_num])

np.random.shuffle(data)


In [27]:
len(data)

6400

In [28]:
from sklearn.model_selection import StratifiedShuffleSplit

In [29]:
X = []
y = []

for features, label in data:
    X.append(features)
    y.append(label)

X = np.array(X)
y = np.array(y)
# out = np.divide(X,255)

y = to_categorical(y, num_classes=len(categories))

n_splits = 1
test_size = 0.2

stratified_split = StratifiedShuffleSplit(n_splits=n_splits,test_size=test_size,random_state=43)

for train_index, test_index in stratified_split.split(X,y):
  X_t, X_test = X[train_index], X[test_index]
  y_t, y_test = y[train_index], y[test_index]

In [30]:
len(X_t)


5120

In [31]:
n_splits = 1
test_size = 0.2

stratified_split = StratifiedShuffleSplit(n_splits=n_splits,test_size=test_size,random_state=43)
# X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
for train_index, val_index in stratified_split.split(X_t,y_t):
  X_train, X_val = X_t[train_index], X_t[val_index]
  y_train, y_val = y_t[train_index], y_t[val_index]

In [32]:
from collections import Counter

train_class_counts = Counter(np.argmax(y_train, axis=1))

val_class_counts = Counter(np.argmax(y_val, axis=1))

test_class_counts = Counter(np.argmax(y_test, axis=1))

print("Training Set Class Counts:")
for class_label, count in train_class_counts.items():
    print(f"Class {class_label}: {count} samples")

print("\nValidation Set Class Counts:")
for class_label, count in val_class_counts.items():
    print(f"Class {class_label}: {count} samples")

print("\nTest Set Class Counts:")
for class_label, count in test_class_counts.items():
    print(f"Class {class_label}: {count} samples")


Training Set Class Counts:
Class 0: 1433 samples
Class 1: 574 samples
Class 3: 2048 samples
Class 2: 41 samples

Validation Set Class Counts:
Class 0: 359 samples
Class 3: 512 samples
Class 1: 143 samples
Class 2: 10 samples

Test Set Class Counts:
Class 0: 448 samples
Class 3: 640 samples
Class 1: 179 samples
Class 2: 13 samples


In [33]:
print(len(X_train),len(X_test),len(X_val),len(X_t))

4096 1280 1024 5120


In [34]:
from keras.optimizers import Adam
from keras.callbacks import LearningRateScheduler

def lr_schedule(epoch,lr):
    if epoch < 8:
        return lr
    else:
        return 0.95*lr

In [35]:
lr_scheduler = LearningRateScheduler(lr_schedule)

In [36]:
optimizer = Adam(learning_rate=0.00095)

In [37]:
from keras import layers
from keras import models

In [38]:
import tensorflow as tf

In [39]:
def create_metrics():
    """
    Creates instances of various metrics for evaluating model performance.

    Returns:
        tuple: A tuple of the metrics: (recall, precision, f1_score, auc, sensitivity, specificity, mcc).
    """
    @tf.function
    def f1_score(y_true, y_pred):
        """
        Calculates the F1 score.

        Args:
            y_true: True labels.
            y_pred: Predicted labels.

        Returns:
            Tensor: F1 score.
        """
        tp = true_positive(y_true, y_pred)
        fp = false_positive(y_true, y_pred)
        fn = false_negative(y_true, y_pred)

        precision = tp / (tp + fp + K.epsilon())
        recall = tp / (tp + fn + K.epsilon())

        return 2 * (precision * recall) / (precision + recall + K.epsilon())

    @tf.function
    def specificity(y_true, y_pred):
        """
        Calculates the specificity.

        Args:
            y_true: True labels.
            y_pred: Predicted labels.

        Returns:
            Tensor: Specificity.
        """
        tn = true_negative(y_true, y_pred)
        fp = false_positive(y_true, y_pred)

        return tn / (tn + fp + K.epsilon())

    @tf.function
    def sensitivity(y_true, y_pred):
        """
        Calculates the sensitivity.

        Args:
            y_true: True labels.
            y_pred: Predicted labels.

        Returns:
            Tensor: Sensitivity.
        """
        tp = true_positive(y_true, y_pred)
        fn = false_negative(y_true, y_pred)

        return tp / (tp + fn + K.epsilon())

    @tf.function
    def mcc(y_true, y_pred):
        """
        Calculates the Matthews correlation coefficient (MCC).

        Args:
            y_true: True labels.
            y_pred: Predicted labels.

        Returns:
            Tensor: Matthews correlation coefficient.
        """
        tp = true_positive(y_true, y_pred)
        tn = true_negative(y_true, y_pred)
        fp = false_positive(y_true, y_pred)
        fn = false_negative(y_true, y_pred)

        numerator = (tp * tn - fp * fn)
        denominator = K.sqrt((tp + fp) * (tp + fn) * (tn + fp) * (tn + fn))

        return numerator / (denominator + K.epsilon())

    @tf.function
    def true_positive(y_true, y_pred):
        """
        Calculates the number of true positives.

        Args:
            y_true: True labels.
            y_pred: Predicted labels.

        Returns:
            Tensor: Number of true positives.
        """
        y_pred_pos, _, y_pos, _ = calculate_confusion_matrix(y_true, y_pred)
        return K.sum(y_pos * y_pred_pos)

    @tf.function
    def false_positive(y_true, y_pred):
        """
        Calculates the number of false positives.

        Args:
            y_true: True labels.
            y_pred: Predicted labels.

        Returns:
            Tensor: Number of false positives.
        """
        y_pred_pos, _, _, y_neg = calculate_confusion_matrix(y_true, y_pred)
        return K.sum(y_neg * y_pred_pos)

    @tf.function
    def false_negative(y_true, y_pred):
        """
        Calculates the number of false negatives.

        Args:
            y_true: True labels.
            y_pred: Predicted labels.

        Returns:
            Tensor: Number of false negatives.
        """
        _, y_pred_neg, y_pos, _ = calculate_confusion_matrix(y_true, y_pred)
        return K.sum(y_pos * y_pred_neg)

    @tf.function
    def true_negative(y_true, y_pred):
        """
        Calculates the number of true negatives.

        Args:
            y_true: True labels.
            y_pred: Predicted labels.

        Returns:
            Tensor: Number of true negatives.
        """
        _, y_pred_neg, _, y_neg = calculate_confusion_matrix(y_true, y_pred)
        return K.sum(y_neg * y_pred_neg)

    @tf.function
    def calculate_confusion_matrix(y_true, y_pred):
        """
        Calculates the components of the confusion matrix.

        Args:
            y_true: True labels.
            y_pred: Predicted labels.

        Returns:
            tuple: A tuple containing the components of the confusion matrix: (y_pred_pos, y_pred_neg, y_pos, y_neg).
        """
        y_pred_pos = K.round(K.clip(y_pred, 0, 1))
        y_pred_neg = 1 - y_pred_pos
        y_pos = K.round(K.clip(y_true, 0, 1))
        y_neg = 1 - y_pos
        return y_pred_pos, y_pred_neg, y_pos, y_neg

    precision_metric = Precision()
    recall_metric = Recall()
    f1_score_metric = f1_score
    specificity_metric = specificity
    sensitivity_metric = sensitivity
    mcc_metric = mcc
    auc_metric = AUC()

    return (
        precision_metric,
        recall_metric,
        f1_score_metric,
        specificity_metric,
        sensitivity_metric,
        mcc_metric,
        auc_metric,
    )

In [40]:
# @title 2D CNN Model Paper Architecture
model = Sequential()
inputs = layers.Input(shape=(100, 100, 3), name="Input")

# x = layers.BatchNormalization()(inputs)
x = layers.Conv2D(32, (3, 3), activation='relu')(inputs)
p1 = layers.MaxPooling2D(pool_size=(2, 2))(x)
# x = layers.BatchNormalization()(p1)

x = layers.Conv2D(64, (3, 3), activation='relu')(p1)
p2 = layers.MaxPooling2D(pool_size=(2, 2))(x)
# x = layers.BatchNormalization()(p2)

x = layers.Conv2D(64, (3, 3), activation='relu')(p2)
p3 =  layers.MaxPooling2D(pool_size=(2, 2))(x)
# c1 = layers.BatchNormalization()(p3)
x = layers.Flatten()(p3)
# model.add(Flatten()(p3))
# print(model.layers[-1].output_shape)

model.add(layers.Dense(128, activation='relu'))
# x = layers.Dense(4, activation='relu')(x)


outputs = layers.Dense(len(categories), activation='softmax')(x)
model = models.Model(inputs=inputs, outputs=outputs)

In [41]:
from tensorflow.keras.utils import plot_model
plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)

In [42]:
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Input (InputLayer)          [(None, 100, 100, 3)]     0         
                                                                 
 conv2d_3 (Conv2D)           (None, 98, 98, 32)        896       
                                                                 
 max_pooling2d_3 (MaxPoolin  (None, 49, 49, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_4 (Conv2D)           (None, 47, 47, 64)        18496     
                                                                 
 max_pooling2d_4 (MaxPoolin  (None, 23, 23, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_5 (Conv2D)           (None, 21, 21, 64)        3692

In [43]:
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy',create_metrics()])

In [44]:
from keras.callbacks import ReduceLROnPlateau, LambdaCallback
from keras import backend as K
lr_history = []

def record_lr(epoch, logs):
    lr_history.append(model.optimizer.lr.numpy())

lr_recorder = LambdaCallback(on_epoch_end=record_lr)


history = model.fit(
    X_train, y_train,
    batch_size=64,
    validation_data=(X_val, y_val),
    epochs=20, # or however many epochs you wish to train for
    callbacks=[lr_scheduler]
)


Epoch 1/20
64/64 [==============================] - 11s 39ms/step - loss: 12.4435 - accuracy: 0.4429 - precision_1: 0.4471 - recall_1: 0.3445 - f1_score: 0.3844 - specificity: 0.8580 - sensitivity: 0.3445 - mcc: 0.2281 - auc_1: 0.6903 - val_loss: 1.0108 - val_accuracy: 0.5117 - val_precision_1: 0.5781 - val_recall_1: 0.2529 - val_f1_score: 0.3511 - val_specificity: 0.9385 - val_sensitivity: 0.2529 - val_mcc: 0.2655 - val_auc_1: 0.7991 - lr: 9.5000e-04
Epoch 2/20
64/64 [==============================] - 2s 29ms/step - loss: 0.9633 - accuracy: 0.5493 - precision_1: 0.6178 - recall_1: 0.3745 - f1_score: 0.4635 - specificity: 0.9228 - sensitivity: 0.3745 - mcc: 0.3594 - auc_1: 0.8250 - val_loss: 0.9210 - val_accuracy: 0.5713 - val_precision_1: 0.6672 - val_recall_1: 0.4170 - val_f1_score: 0.5121 - val_specificity: 0.9307 - val_sensitivity: 0.4170 - val_mcc: 0.4140 - val_auc_1: 0.8422 - lr: 9.5000e-04
Epoch 3/20
64/64 [==============================] - 2s 27ms/step - loss: 0.8715 - accuracy

In [45]:
result = model.evaluate(X_test, y_test)

40/40 [==============================] - 1s 5ms/step - loss: 0.3280 - accuracy: 0.8828 - precision_1: 0.8901 - recall_1: 0.8734 - f1_score: 0.8814 - specificity: 0.9641 - sensitivity: 0.8734 - mcc: 0.8426 - auc_1: 0.9801


In [46]:
from sklearn.metrics import classification_report
import numpy as np

# Make predictions
y_pred = model.predict(X_test)

y_pred_classes = np.argmax(y_pred, axis=1)
y_val_classes = np.argmax(y_test, axis=1)

# Generate a classification report
report = classification_report(y_val_classes, y_pred_classes, target_names=categories)

print(report)


40/40 [==============================] - 0s 3ms/step
                  precision    recall  f1-score   support

VeryMildDemented       0.86      0.86      0.86       448
    MildDemented       0.88      0.81      0.84       179
ModerateDemented       1.00      0.77      0.87        13
     NonDemented       0.90      0.92      0.91       640

        accuracy                           0.88      1280
       macro avg       0.91      0.84      0.87      1280
    weighted avg       0.88      0.88      0.88      1280



32/32 [==============================] - 0s 3ms/step


In [47]:
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import numpy as np

# Make predictions on the validation set
y_pred = model.predict(X_val)

# Convert predictions and true labels to class indices
y_pred_classes = np.argmax(y_pred, axis=1)
y_val_classes = np.argmax(y_val, axis=1)

# Calculate the confusion matrix
cm = confusion_matrix(y_val_classes, y_pred_classes)

# Calculate the accuracy for each class
class_accuracies = cm.diagonal() / cm.sum(axis=1)

# Plotting
plt.bar(categories, class_accuracies)
plt.xlabel('Class')
plt.ylabel('Accuracy')
plt.title('Accuracy for Each Class')
plt.ylim([0, 1])  # Set the limit of y-axis to [0,1] for percentage
plt.show()


In [48]:
def plot_metrics(hist, metric_name_list):
    """
    Plots the given metrics for the training and validation sets.

    Args:
        hist: a Keras History object containing the training history
        metric_name_list: a list of strings representing the names of the metrics to plot

    Returns:
        None
    """
    num_metrics = len(metric_name_list)

    figure, axes = plt.subplots(num_metrics, figsize=(10, 6 * num_metrics))

    rng = range(1, len(hist.history[metric_name_list[0]]) + 1)

    for ax, metric_n in zip(axes, metric_name_list):
        metric = hist.history[metric_n]
        v_metric = hist.history[f"val_{metric_n}"]

        ax.plot(rng, metric, label=metric_n)
        ax.plot(rng, v_metric, label=f"val_{metric_n}")
        ax.legend()
        ax.set_xlabel("Epochs")

        if metric_n in ("auc", "mcc"):
            ax.set_ylabel(metric_n.upper())
            ax.set_title(metric_n.upper() + " vs Epochs")
        else:
            ax.set_ylabel(metric_n.capitalize())
            ax.set_title(metric_n.capitalize() + " vs Epochs")

        max_loss = max(max(metric), max(v_metric))
        min_loss = min(min(metric), min(v_metric))
        y_max = math.ceil(max_loss)

        if min_loss > 0 or max_loss > 1:
            ax.set_ylim(0, y_max)
        else:
            ax.set_ylim(min_loss, y_max)

        ax.grid(True, linestyle='--', alpha=0.5)
        ax.set_xlim(1, len(metric))

    plt.tight_layout()

    plt.show()

In [49]:
metric_names = ["loss", "accuracy","f1_score",
                "specificity", "sensitivity", "mcc"]
plot_metrics(history, metric_names)